In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='XPTO',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [2]:
db('''
SELECT
    *
FROM 
    pizza

''')

Executando query:
('calabresa', 100.0)
('incoerente', 200.0)
('legumes', 100.0)
('mussarela', 100.0)
('pão', 100.0)


### 1.a)
Liste as 3 pizzas que dão mais lucro. O lucro de uma pizza é o seu preço de venda menos o
custo total dos ingredientes nela utilizados (não se esqueça de que cada ingrediente da pizza tem uma
especificação de quantidade a ser utilizada).

In [7]:
db('''
SELECT
    pizza, preco - SUM(preco_unitario*quantidade) as lucro
FROM 
    pizza
    INNER JOIN ingredientepizza USING(pizza)
    INNER JOIN ingrediente USING(ingrediente)
GROUP BY
    pizza
ORDER BY
    lucro DESC
LIMIT 
    3
''')

Executando query:
('pão', 95.0)
('incoerente', 83.0)
('legumes', 78.0)


### 1.b) 
Construa uma tabela temporária que liste todas as pizzas e a quantidade de ingredientes nela

In [18]:
db('''
    DROP TABLE IF EXISTS QtdeTotal;
''')

db('''
CREATE TEMPORARY TABLE QtdeTotal

SELECT
    pizza, COUNT(ingrediente) as qtde_total
FROM 
    ingredientepizza
GROUP BY
    pizza
''')

db('''
SELECT * FROM QtdeTotal;
''')

Executando query:
Executando query:
Executando query:
('calabresa', 2)
('incoerente', 4)
('legumes', 2)
('mussarela', 2)
('pão', 1)


### 1.c)
Construa uma tabela temporária que liste todas as pizzas a quantidade de ingredientes
veganos nela (ou seja, onde o tipo é “vegana”)

In [19]:
db('''
    DROP TABLE IF EXISTS QtdeVegano;
''')

db('''
CREATE TEMPORARY TABLE QtdeVegano

SELECT
    pizza, COUNT(ingrediente) as qtde_vegano
FROM 
    ingredientepizza
    INNER JOIN ingredientetipo USING(ingrediente)
WHERE
    tipo = 'vegana'
GROUP BY
    pizza
''')

db('''
SELECT * FROM QtdeVegano;
''')

Executando query:
Executando query:
Executando query:
('legumes', 2)
('incoerente', 2)


### 1.d)
Usando as tabelas temporárias dos itens (b) e (c) – mesmo que você não os tenha feito – liste
as pizzas veganas da pizzaria. Uma pizza é vegana se todos os seus ingredientes são veganos.

In [21]:
db('''
SELECT
    pizza
FROM 
    QtdeTotal
    INNER JOIN QtdeVegano USING(pizza)
WHERE
    qtde_total = qtde_vegano
GROUP BY
    pizza
''')

Executando query:
('legumes',)


### 1.e)
Liste os ingredientes e seus tipos, um ingrediente por linha (Dica: use a função
GROUP_CONCAT). O resultado deve ser da seguinte forma:

In [28]:
db('DROP TABLE IF EXISTS ingrediente_tipos')
db('''
CREATE TEMPORARY TABLE ingrediente_tipos
    SELECT 
        ingrediente, 
        CONCAT(
            GROUP_CONCAT(tipo ORDER BY tipo SEPARATOR ", ")
        ) as lista_tipos
    FROM
        ingredientetipo  
    GROUP BY
        ingrediente
''')
db('''
SELECT * FROM ingrediente_tipos;
''')

Executando query:
Executando query:
Executando query:
('abobrinha', 'sem gluten, sem lactose, vegana, vegetariana')
('calabresa', 'sem gluten, sem lactose')
('massa', 'vegetariana')
('massa sem gluten', 'sem gluten, sem lactose, vegana, vegetariana')
('mussarela', 'sem gluten, vegetariana')
('mussarela vegana', 'sem gluten, sem lactose, vegana, vegetariana')
('picanha', 'sem gluten, sem lactose')


### 3.a)

Por que normalizar um design de banco de dados?

In [ ]:
Para facilitar o uso, já inputs e modelos são padronizados
e para facilitar a manutenção e expansão já que a informação é apresentada de forma clara e consistente

### 3.b)

Por que usar um sistema de gerenciamento de banco de dados e não um arquivo simples?

In [ ]:
Pode não caber na RAM!
Existem maneiras eficientes
de armazenar e recuperar dados!
O SGBD já vem com mecanismos
sofisticados de consulta!
Um bom projeto evita redundâncias
gerencia transações e mantém consistência!
Controle de acesso e registro de atividade!
Proteção contra falhas!

### 3.d)

Qual a diferença entre WHERE e HAVING?

In [ ]:
Where é usado antes de agrupamentos e o having depois